In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import norm
from scipy.stats import gstd as geo_std
import datetime
import os
import supertrend_lib as stlib
import sbeta_lib as sbeta
import smacross_lib as smacross
import CAPM_lib as capm
import LR_run_model_lib as LR_run
import LR_model_lib as LR_models
import yfinance as yf
pd.set_option('display.max_rows', 200)
from IPython.display import display, HTML


In [2]:
# symbols_list = eval_df.Stock.values
def get_company_info(symbols_list):
    info_list = []

    for s in symbols_list:
        shandle = yf.Ticker(s)
        info_list.append(shandle.info)

    return info_list
    
# info_list = get_company_info(['aapl','msft'])
# print(info_list)

In [3]:
def add_update(eval_df,values):
    """ Add a a row to dataframe from a list of data points in the same sequence as the columns

    Args:
        eval_df (_type_): DataFrame receiving the new row
        values_list (_type_): A list of values in the same data types and sequance order as the dataframe columns

    Returns:
        _type_: The updated dataframe
    """
    tmp_df = pd.DataFrame([values], columns=eval_df.columns)
    eval_df = pd.concat([eval_df, tmp_df], ignore_index=True)
    return eval_df

In [4]:
def read_stocklist(filename):
    with open(filename) as f:
        stocklist = f.read()
    return stocklist.strip().split(",")


### Write Dataframe as Table to HTML File

In [5]:
from prettytable import PrettyTable

def df_to_html_file(input_df, output_file_name, table_title):
    # Create a PrettyTable object
    table = PrettyTable(input_df.columns.tolist())

    # Add rows to the table
    for _, row in input_df.iterrows():
        table.add_row(row.tolist())

    # Generate the HTML table
    html_table = table.get_html_string()

    # Add table title
    html_table = f'<h2>{table_title}</h2>\n' + html_table

    # Add CSS styling
    css_style = '''
        <style>
        table {
            border-collapse: collapse;
        }
        th, td {
            border: 1px solid black;
            padding: 2px;
        }
        </style>
    '''

    # Combine CSS styling and table content
    html_table = css_style + html_table

    # Save the HTML table to the file
    with open(output_file_name, 'w') as f:
        f.write(html_table)

    # # Open the HTML file and print some information
    # with open(output_file_name, 'r') as f:
    #     file_lines = f.readlines()

    # print(f"Number of lines in the HTML file: {len(file_lines)}")
    # print(f"First few lines of the HTML file: {file_lines[:5]}")
    
    return

In [6]:
def update_models(stock_list):
    ret = LR_models.update_list_of_models(stock_list=stock_list)
    models_df = pd.DataFrame({'Stock' : stock_list,
                              'Model' : [f"Model{i}" for i in ret]})
    
    # print(models_df)
    return models_df

In [7]:
# stocks_from_file = read_stocklist(filename='./stocks_list.txt')
# stocks_from_file = [s.strip().upper() for s in list(stocks_from_file)]
# dfff = update_models(stocks_from_file)
# dfff

In [8]:
# dfff[dfff['Stock'] == 'AAPL']['Model'].values[0]

In [9]:
def run_supertrend(stock,start_date):
    return stlib.supertrend(stock,start_date=start_date)


In [10]:
def init_eval_table():
    columns = ['Last Run'          ,'Stock'             ,'Last Price'     ,'%Std. Dev.','$Std. Dev.','Supertrend Winner' ,'Supertrend Result','ST Signal_Date', 
               'LR Best_Model'     ,'LR Next_Day Recomm', 'SMA Crossed_Up','SMA_X_Date'        ,'SMA FastXSlow'    , 'Beta'         , 
               '%Sharpe Ratio'                  ,'CAPM'              ,'Daily VaR'  ]

    dtypes = ['datetime64[ns]'       , str                , float            ,str      ,str        , bool              , str               , str            , 
              str                  , str                , str              , 'datetime64[ns]'    , str               , float          , 
              float                , float                , float        ]

    # Initialize the DataFrame with empty rows
    eval_df = pd.DataFrame(columns=columns)

    # Convert the column data types
    for col, dtype in zip(columns, dtypes):
        eval_df[col] = eval_df[col].astype(dtype)
        
    return eval_df

In [11]:
def first_date_N_years_ago(years):
    # Calculate start and end dates N years back
    end_date = datetime.date.today() - datetime.timedelta(days=1)
    start_date = end_date.replace(year=end_date.year - years, month=1, day=1)
    return start_date.strftime("%Y-%m-%d")


In [12]:
def load_sp500_list():
    stocks_list_csv = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp_df = pd.DataFrame()
    
    try:
        wiki_data=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies') # Open the link and download S&P company details in a table
        data = wiki_data[0] # All data is stored in first cell
        sp_df = data.sort_values(by=['Symbol'], ascending=True) # Sort the dataframe on ticker in alphabetical ascending order
    except:
        print("Cannot open file", stocks_list_csv)

    # remove the dotted symbols, they are redundant 
    no_dot_symbols = [i for i in sp_df['Symbol'] if i.find('.')==-1]
    sp_df = sp_df[sp_df['Symbol'].isin(no_dot_symbols)]
    
    # returns count_row, count_col, df
    return sp_df.shape[0], sp_df.shape[1], sp_df


In [13]:
m,n,sp_df = load_sp500_list()
# sp_df[sp_df['Symbol'].isin([i for i in sp_df['Symbol'] if i.find('.')>0])]

In [14]:
def get_gstd(stock_data):
    # Calculate daily price changes
    price_changes = stock_data["Close"].pct_change()
    price_changes = price_changes.dropna()

    # Calculate the mean and geometric standard deviation of the daily price changes
    gstd = geo_std(price_changes)

    # Return the mean and geometric standard deviation
    return gstd

def get_sigma(stock_data):
    # Calculate daily price changes
    price_changes = stock_data["Close"].pct_change()

    mu, std = norm.fit(price_changes.dropna())
    return mu, std

In [15]:
def limit_trailing_stop_order_percentage(stock,stock_data,multiple_of_std_dev):
    close_price = stock_data.iloc[-1]['Close']
    mean, std_dev = get_sigma(stock_data)
    std_dev_pct = round(std_dev * 100,2)
    std_dev_dlr = round(close_price*std_dev,2)
    
    # use (std_dev_pct x multiple_of_std_dev) as the % to get the limit price
    
    pct_drop = -round(multiple_of_std_dev * std_dev_pct,2) 
    price_drop = round(close_price * pct_drop/100,2)
    min_trailing_stop_price = close_price + price_drop
    
    limit_price = min_trailing_stop_price + std_dev_dlr
    limit_price = round(limit_price,2)
    
    return close_price, pct_drop, price_drop, min_trailing_stop_price, limit_price, std_dev_pct, std_dev_dlr

In [16]:
def recommendation_table(eval_df,stock_list, lookback_years=1, sma_fast=40, sma_slow=200,run_update_models=False):
    stock_list = [str(s).upper() for s in stock_list]
    # print("Stock List:",stock_list)
    start_date = first_date_N_years_ago(lookback_years)
    today = datetime.datetime.today().strftime("%Y-%m-%d")
    if run_update_models :
        print("Updating Models ...",end='')
        stock_best_model_df = update_models(stock_list=stock_list)
        print("Done!")

    print('Performing Analysis and Recommendations ...',end='')

    for s in stock_list:
        winner,buysell,buysell_date,close_price,stock_data = run_supertrend(s,start_date)
        if(len(stock_data)<2):
            print(f"{s} data not found. Skipping!",end=",")
            continue
        
        print(f"{s}",end=',')
        mean, std_dev = get_sigma(stock_data)
        # std_dev = get_gstd(stock_data)
        std_dev_pct = round(std_dev * 100,2)
        std_dev_dlr = round(close_price*std_dev,2)
        sma_sig, sma_date, fastXslow = smacross.sma_xing(stock_data,sma_fast,sma_slow)
        beta, market_data = sbeta.get_beta(stock_data)
        CAPM, VaR, Sharpe = capm.CAPM_VaR(stock_data=stock_data,market_data=market_data,bond_mat_duration = lookback_years,stock_beta=beta)
        Sharpe = round(Sharpe,2)
        LR_recommend = LR_run.get_recommendation(stock=s,lookback=lookback_years)
        LR_recommend_str = f"{LR_recommend[0]},{LR_recommend[1]},{LR_recommend[2]}"
        
        # columns = ['Last_Run'          ,'Stock'             ,'Last_Price'     ,'%Std. Dev.','$Std. Dev.','Supertrend_Winner' ,'Supertrend_Result','ST_Signal_Date', 
        #            'LR_Best_Model'     ,'LR_Next_Day_Recomm', 'SMA_Crossed_Up','SMA_X_Date'             ,'SMA_FastXSlow'     , 'Beta'         , 
        #            'SharpeRatio'                  ,'CAPM'              ,'Daily_VaR'  ]
    
        if run_update_models:
            model_str = stock_best_model_df[stock_best_model_df['Stock'] == s].Model.values[0]
        else:
            model_str = 'N/A'
            
        new_row = [today           , s                  , close_price          ,f"+/-{std_dev_pct}%",f"+/-${std_dev_dlr}",    winner            , buysell         ,    buysell_date ,    
                   model_str       , LR_recommend_str   , sma_sig              , sma_date           , fastXslow          ,    beta              , 
                   Sharpe          , CAPM            , VaR  ]    
        
        eval_df = add_update(eval_df=eval_df,values = new_row)
        
    print("Done!")
    return eval_df

In [17]:
def recommend_selling_strategy(lookback_years,stock_list,multiple_of_std_dev):
    """
    This function is used to evaluate the selling strategy of the stocks.
    It will return a DataFrame with the following columns:
    - Stock
    - $Last Price
    - Multiple of Std.Dev. Drop
    - %Std. Dev.
    - $Std. Dev.
    - %Trailing Stop Drop
    - $Trailing Stop Drop
    - $Min. Trailing Stop Price
    - $Recomm. Limit Price Order
    """

    # Initialize the DataFrame with empty rows
    start_date = first_date_N_years_ago(lookback_years)

    columns = ['Stock'          ,'$Last Price' , 'Multiple of Std Drop'    ,'%Std. Dev.','$Std. Dev.','%Trailing Stop Drop' , '$Trailing Stop Drop','$Min. Trailing Stop Price','$Recomm. Limit Price Order']
    dtypes = [str               , float        ,float                      ,float        , float     , float                , float                , float              , float]

    # Initialize the DataFrame with empty rows
    sell_orders_recomm = pd.DataFrame(columns=columns)

    # Convert the column data types
    for col, dtype in zip(columns, dtypes):
        sell_orders_recomm[col] = sell_orders_recomm[col].astype(dtype)


    for s in stock_list:
        winner,buysell,buysell_date,close_price,stock_data = run_supertrend(s,start_date)
        if(len(stock_data)<2):
            print(f"{s.upper()} data not found. Skipping!",end=",")
            continue
        else:
            close_price, pct_drop, price_drop, min_trailing_stop_price, limit_price, std_dev_pct, std_dev_dlr = \
                limit_trailing_stop_order_percentage(stock=s,stock_data=stock_data,multiple_of_std_dev=multiple_of_std_dev)
            # print(f"{s.upper()} Last Price: ${round(close_price,2)}, %Multiple of Std.Dev. Drop %:{multiple_of_std_dev}, % Trailing Drop:{pct_drop}%, \
            #       Limit Price:${limit_price}, Min Price:${round(min_trailing_stop_price,2)}, Std %:{std_dev_pct}, Std $:{std_dev_dlr}")
            
            new_row = [s.upper()  , f"${round(close_price,2)}"    , f"{multiple_of_std_dev}x"     ,f"+/-{std_dev_pct}%",f"+/-${std_dev_dlr}",    f"{pct_drop}%",    f"${price_drop}"   ,  f"${round(min_trailing_stop_price,2)}"  , f"${limit_price}"]    
            sell_orders_recomm = add_update(eval_df=sell_orders_recomm,values = new_row)
            
    return sell_orders_recomm


In [18]:
def eval_all_sp500(lookback_years = 1,sma_fast = 50, sma_slow = 200, regenerate_models = False ):
    n,m,sp_df = load_sp500_list()
    stocks_sp500 = sp_df.Symbol.values
    stocks_sp500 = [s.strip().upper() for s in list(stocks_sp500)]

    eval_df = init_eval_table()
    eval_df = recommendation_table(eval_df,
                                stock_list=stocks_sp500, 
                                lookback_years=lookback_years, 
                                sma_fast=sma_fast, 
                                sma_slow=sma_slow,run_update_models=regenerate_models)
    
    return eval_df

In [19]:
def eval_list_from_file(filename='./stocks_list.txt',lookback_years = 1,sma_fast = 50, sma_slow = 200 ,regenerate_models = False):
    stocks_from_file = read_stocklist(filename=filename)
    stocks_from_file = [s.strip().upper() for s in list(stocks_from_file)]
    today = datetime.datetime.today().strftime("%Y-%m-%d")

    eval_df = init_eval_table()
    eval_df = recommendation_table(eval_df,
                                stock_list=stocks_from_file, 
                                lookback_years=lookback_years, 
                                sma_fast=sma_fast, 
                                sma_slow=sma_slow,run_update_models=regenerate_models)
    
    return eval_df

In [20]:
def screen_for_buys(eval_df, ignore_supertrend_winners=False):
        if not ignore_supertrend_winners:
                buys_df = eval_df[ (eval_df['Supertrend Winner']==True) &  
                        (eval_df['Supertrend Result']=='Buy') & 
                        (eval_df['LR Next_Day Recomm'] == 'Buy,Buy,Buy') &
                        (eval_df['SMA Crossed_Up']=='Buy')].sort_values(by=['Supertrend Winner','Supertrend Result',
                                                                            'ST Signal_Date','SMA Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])
        else:
                buys_df = eval_df[ (eval_df['Supertrend Result']=='Buy') & 
                        (eval_df['LR Next_Day Recomm'] == 'Buy,Buy,Buy') &
                        (eval_df['SMA Crossed_Up']=='Buy')].sort_values(by=['Supertrend Winner','Supertrend Result',
                                                                            'ST Signal_Date','SMA Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])            
        
        return buys_df

In [21]:
def screen_for_sells(eval_df, ignore_supertrend_winners=False):
        if not ignore_supertrend_winners:
                sells_df = eval_df[ (eval_df['Supertrend Winner']==True) &  
                        ((eval_df['Supertrend Result']=='Sell') |
                        (eval_df['LR Next_Day Recomm'] == 'Sell,Sell,Sell') |
                        (eval_df['SMA Crossed_Up']=='Sell')) ].sort_values(by=['Supertrend Winner','Supertrend Result',
                                                                            'ST Signal_Date','SMA Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])
        else:
                sells_df = eval_df[ (eval_df['Supertrend Result']=='Sell') |
                        (eval_df['LR Next_Day Recomm'] == 'Sell,Sell,Sell') |
                        (eval_df['SMA Crossed_Up']=='Sell')].sort_values(by=['Supertrend Winner','Supertrend Result',
                                                                            'ST Signal_Date','SMA Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])            
        
        return sells_df

In [22]:
results_dir = './eval_results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
fname = f'{results_dir}/Eval_Results_{datetime.datetime.today().strftime("%Y_%m_%dT%I%M%S%p")}.csv'

### Evaluate Stocks List

In [23]:
regenerate_models = True
# symbols_file = './stocks_portfolio.txt'
symbols_file = './stocks_list.txt'
# symbols_file = './sectors_etfs.txt'
stocks_list = read_stocklist(symbols_file)  

eval_df = eval_list_from_file(filename=symbols_file,lookback_years=2,regenerate_models=regenerate_models)
# eval_df_from_file = eval_df
# eval_df = eval_all_sp500(lookback_years=2, regenerate_models=regenerate_models)
eval_df.to_csv(fname)

Updating Models ...QQQ,AAPL,MSFT,NVDA,AMD,INTC,META,IBM,QCOM,MU,TSLA,SPY,STX,GOOGL,AMZN,SQ,COIN,MRVL,ORCL,NFLX,PLTR,AVGO,DE,CDNS,ADBE,NOW,SMCI,KLAC,TSM,ASML,LRCX,ON,TXN,AMAT,JBL,ADI,MCHP,SWKS,HON,DIA,PANW,Done!
Performing Analysis and Recommendations ...QQQ,AAPL,MSFT,NVDA,AMD,INTC,META,IBM,QCOM,MU,TSLA,SPY,STX,GOOGL,AMZN,SQ,COIN,MRVL,ORCL,NFLX,PLTR,AVGO,DE,CDNS,ADBE,NOW,SMCI,KLAC,TSM,ASML,LRCX,ON,TXN,AMAT,JBL,ADI,MCHP,SWKS,HON,DIA,PANW,Done!


### All Results

In [24]:
print(f"{len(eval_df)} Stocks:")
display(HTML(eval_df.sort_values(['ST Signal_Date','SMA_X_Date'], ascending=False).to_html(index=False)))
ret = df_to_html_file(eval_df,"/var/www/html/home/viewable_pages/stock_evaluation.php","Latest Stock Evaluation: All") 

41 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,SQ,62.60,+/-4.33%,+/-$2.71,False,Sell,2023-08-04,Model2,"Buy,Buy,Buy",Buy,2023-07-31,"(50, 200)",2.5533,-2.89,10.30,-10.23
2023-08-08,COIN,87.92,+/-5.94%,+/-$5.23,False,Sell,2023-08-04,Model2,"Sell,Buy,Buy",Buy,2023-06-30,"(50, 200)",2.7425,-1.11,2.59,-14.35
2023-08-08,AAPL,179.94,+/-1.82%,+/-$3.28,True,Sell,2023-08-04,Model2,"Buy,Buy,Buy",Buy,2023-03-23,"(50, 200)",1.2874,2.57,7.53,-4.37
2023-08-08,MCHP,84.81,+/-2.56%,+/-$2.17,False,Sell,2023-08-04,Model2,"Buy,Buy,Buy",Buy,2022-12-07,"(50, 200)",1.6759,1.70,8.65,-6.14
2023-08-08,QCOM,117.82,+/-2.49%,+/-$2.94,False,Sell,2023-08-03,Model2,"Buy,Buy,Buy",Buy,2023-08-04,"(50, 200)",1.5288,-1.08,8.09,-5.89
2023-08-08,TXN,166.82,+/-1.8%,+/-$3.0,False,Sell,2023-08-02,Model2,"Sell,Sell,Sell",Buy,2023-07-13,"(50, 200)",1.1784,0.05,7.33,-4.28
2023-08-08,STX,67.12,+/-2.53%,+/-$1.7,False,Buy,2023-07-28,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(50, 200)",1.2141,1.19,7.58,-6.03
2023-08-08,INTC,34.99,+/-2.29%,+/-$0.8,False,Buy,2023-07-28,Model2,"Sell,Sell,Sell",Buy,2023-05-09,"(50, 200)",1.2500,-2.14,7.47,-5.41
2023-08-08,HON,187.73,+/-1.36%,+/-$2.56,False,Sell,2023-07-27,Model1,"Buy,Buy,Buy",Sell,2023-08-02,"(50, 200)",0.8163,-1.85,6.65,-3.20
2023-08-08,MSFT,330.11,+/-1.82%,+/-$6.02,False,Sell,2023-07-27,Model2,"Buy,Buy,Buy",Buy,2023-03-21,"(50, 200)",1.2624,3.41,7.69,-4.38


### Exit Strategy

In [45]:
lookback_years = 2
# my_stock_list = ['amd','jbl','aapl','nvda','msft','googl','amzn','nflx','meta','adbe','cdns','de','avgo','orcl']
my_stock_list = stocks_list
multiple_of_std_dev = 1.5 # Stop Loss as percentage of Std.Dev.
sell_orders_recomm = recommend_selling_strategy(lookback_years,my_stock_list,multiple_of_std_dev)

print("Order Type: Limit Trailing Stop Loss Percent (%) Orders ONLY:  ")
print("=============================================================")
print(f"{len(sell_orders_recomm)} Stocks:")
display(HTML(sell_orders_recomm.to_html(index=False)))
ret = df_to_html_file(sell_orders_recomm,"/var/www/html/home/viewable_pages/exit_strategy.php","Stock Positions Recommended Exit Strategies <br>Order Type: Limit Trailing Stop Loss Percent (%) Orders ONLY") 

Order Type: Limit Trailing Stop Loss Percent (%) Orders ONLY:  
41 Stocks:


Stock,$Last Price,Multiple of Std Drop,%Std. Dev.,$Std. Dev.,%Trailing Stop Drop,$Trailing Stop Drop,$Min. Trailing Stop Price,$Recomm. Limit Price Order
QQQ,$372.0,1.5x,+/-1.56%,+/-$5.8,-2.34%,$-8.7,$363.3,$369.1
AAPL,$179.8,1.5x,+/-1.82%,+/-$3.28,-2.73%,$-4.91,$174.89,$178.17
MSFT,$326.05,1.5x,+/-1.82%,+/-$5.94,-2.73%,$-8.9,$317.15,$323.09
NVDA,$446.64,1.5x,+/-3.47%,+/-$15.48,-5.2%,$-23.23,$423.41,$438.89
AMD,$113.23,1.5x,+/-3.29%,+/-$3.72,-4.94%,$-5.59,$107.64,$111.36
INTC,$35.02,1.5x,+/-2.29%,+/-$0.8,-3.44%,$-1.2,$33.82,$34.62
META,$312.64,1.5x,+/-3.12%,+/-$9.74,-4.68%,$-14.63,$298.01,$307.75
IBM,$145.9,1.5x,+/-1.4%,+/-$2.04,-2.1%,$-3.06,$142.84,$144.88
QCOM,$117.66,1.5x,+/-2.49%,+/-$2.93,-3.74%,$-4.4,$113.26,$116.19
MU,$67.65,1.5x,+/-2.69%,+/-$1.82,-4.04%,$-2.73,$64.92,$66.74


### Send to Printer

In [26]:
if symbols_file == './stocks_list.txt':
    eval_df.to_csv("./stocks_eval_df_table.csv")
elif symbols_file == './sectors_etfs.txt':
    eval_df.to_csv("./sectors_etfs_eval_df_table.csv")
    
print_it = False

def send_df_to_printer(df):
    pdf_filename = "full_eval_df_table.pdf"

    fig, ax =plt.subplots(figsize=(12,4))
    ax.axis('tight')
    ax.axis('off')
    the_table = ax.table(cellText=df.values,colLabels=df.columns,loc='top')

    pp = PdfPages(pdf_filename)
    pp.savefig(fig, bbox_inches='tight')
    pp.close()

    os.system("lpr "+pdf_filename)
    
if (print_it):
    send_df_to_printer(eval_df)

### Buy, Buy, and more Buys

In [27]:
buys_eval_df= screen_for_buys(eval_df=eval_df,ignore_supertrend_winners=False)
buys_eval_df = buys_eval_df.sort_values('%Sharpe Ratio',ascending=True).sort_values('Daily VaR',ascending=False)
print(f"{len(buys_eval_df)} Stocks:")
display(HTML(buys_eval_df.to_html(index=False)))
ret = df_to_html_file(buys_eval_df,"/var/www/html/home/viewable_pages/buybuybuy.php","Buy, Buy, Buy Rated Stocks") 

3 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,JBL,108.37,+/-1.99%,+/-$2.16,True,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2022-10-03,"(50, 200)",1.2523,7.19,7.67,-4.87
2023-08-08,ADBE,520.46,+/-2.34%,+/-$12.18,True,Buy,2023-05-19,Model2,"Buy,Buy,Buy",Buy,2023-03-21,"(50, 200)",1.4736,0.85,7.95,-5.65
2023-08-08,NVDA,446.38,+/-3.47%,+/-$15.47,True,Buy,2023-01-12,Model2,"Buy,Buy,Buy",Buy,2023-01-24,"(50, 200)",2.1795,6.51,9.82,-8.46


### Buy, But, and more Buys (Not necessarily ST winners)

In [28]:
buys_eval_df= screen_for_buys(eval_df=eval_df,ignore_supertrend_winners=True)
buys_eval_df = buys_eval_df.sort_values('%Sharpe Ratio',ascending=True).sort_values('Daily VaR',ascending=False)
print(f"{len(buys_eval_df)} Stocks:")
display(HTML(buys_eval_df.to_html(index=False)))
ret = df_to_html_file(buys_eval_df,"/var/www/html/home/viewable_pages/buybuybuy2.php","Buy, Buy, Buy Rated Stocks (Not necessarily ST winners)") 

12 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,ADI,186.52,+/-1.98%,+/-$3.69,False,Buy,2023-06-12,Model2,"Buy,Buy,Buy",Buy,2022-12-23,"(50, 200)",1.2949,1.82,7.56,-4.73
2023-08-08,JBL,108.37,+/-1.99%,+/-$2.16,True,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2022-10-03,"(50, 200)",1.2523,7.19,7.67,-4.87
2023-08-08,ADBE,520.46,+/-2.34%,+/-$12.18,True,Buy,2023-05-19,Model2,"Buy,Buy,Buy",Buy,2023-03-21,"(50, 200)",1.4736,0.85,7.95,-5.65
2023-08-08,SWKS,108.72,+/-2.42%,+/-$2.63,False,Buy,2023-06-12,Model2,"Buy,Buy,Buy",Buy,2023-02-08,"(50, 200)",1.5268,-1.85,8.05,-5.72
2023-08-08,STX,67.12,+/-2.53%,+/-$1.7,False,Buy,2023-07-28,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(50, 200)",1.2141,1.19,7.58,-6.03
2023-08-08,MU,69.38,+/-2.69%,+/-$1.87,False,Buy,2023-07-26,Model2,"Buy,Buy,Buy",Buy,2023-03-09,"(50, 200)",1.5015,0.04,8.02,-6.38
2023-08-08,KLAC,498.68,+/-2.73%,+/-$13.63,False,Buy,2023-05-15,Model2,"Buy,Buy,Buy",Buy,2022-12-14,"(50, 200)",1.7169,4.23,8.48,-6.62
2023-08-08,AMAT,148.06,+/-2.84%,+/-$4.2,False,Buy,2023-05-15,Model2,"Buy,Buy,Buy",Buy,2023-01-10,"(50, 200)",1.8025,3.49,8.94,-6.86
2023-08-08,LRCX,697.38,+/-2.93%,+/-$20.4,False,Buy,2023-03-30,Model2,"Buy,Buy,Buy",Buy,2023-01-17,"(50, 200)",1.8100,2.63,8.96,-7.06
2023-08-08,ON,101.62,+/-3.26%,+/-$3.31,False,Buy,2023-05-01,Model1,"Buy,Buy,Buy",Buy,2022-08-25,"(50, 200)",1.9852,6.33,9.10,-7.96


### Save the Buys

In [29]:
eval_df.to_csv("./stocks_buys_eval_df_table.csv")

### Time to Sell!

In [30]:
sells_eval_df= screen_for_sells(eval_df=eval_df,ignore_supertrend_winners=True)
print(f"{len(sells_eval_df)} Stocks:")
display(HTML(sells_eval_df.to_html(index=False)))
ret = df_to_html_file(sells_eval_df,"/var/www/html/home/viewable_pages/time_to_sell.php","Time to Sell") 

23 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,SMCI,349.32,+/-3.74%,+/-$13.07,True,Buy,2023-07-10,Model2,"Sell,Sell,Sell",None,None,"(50, 200)",1.3871,11.23,7.98,-9.12
2023-08-08,DE,431.31,+/-1.99%,+/-$8.6,True,Buy,2023-06-07,Model1,"Buy,Buy,Buy",Sell,2023-05-10,"(50, 200)",0.8351,3.57,6.57,-4.82
2023-08-08,META,312.49,+/-3.12%,+/-$9.74,True,Buy,2022-11-10,Model2,"Sell,Sell,Sell",Buy,2023-03-02,"(50, 200)",1.6822,1.71,8.66,-7.64
2023-08-08,AAPL,179.94,+/-1.82%,+/-$3.28,True,Sell,2023-08-04,Model2,"Buy,Buy,Buy",Buy,2023-03-23,"(50, 200)",1.2874,2.57,7.53,-4.37
2023-08-08,ASML,680.31,+/-2.7%,+/-$18.36,True,Sell,2023-07-20,Model2,"Buy,Buy,Buy",Buy,2022-12-23,"(50, 200)",1.8007,2.35,8.68,-6.47
2023-08-08,PANW,210.98,+/-2.6%,+/-$5.48,True,Sell,2023-07-12,Model1,"Sell,Sell,Sell",Buy,2023-03-20,"(50, 200)",1.1747,4.07,7.30,-6.23
2023-08-08,ORCL,115.06,+/-1.8%,+/-$2.07,True,Sell,2023-06-26,Model2,"Buy,Buy,Buy",Buy,2022-12-09,"(50, 200)",0.8725,5.01,6.66,-4.32
2023-08-08,INTC,34.99,+/-2.29%,+/-$0.8,False,Buy,2023-07-28,Model2,"Sell,Sell,Sell",Buy,2023-05-09,"(50, 200)",1.2500,-2.14,7.47,-5.41
2023-08-08,CDNS,228.89,+/-2.24%,+/-$5.13,False,Buy,2023-05-18,Model2,"Sell,Sell,Sell",Buy,2023-01-10,"(50, 200)",1.3836,3.86,7.77,-5.41
2023-08-08,PLTR,16.89,+/-4.41%,+/-$0.74,False,Buy,2023-05-09,Model1,"Sell,Sell,Sell",Buy,2023-04-05,"(50, 200)",1.8411,0.45,8.76,-10.55


### All Roads Lead to UP & Safe

In [31]:
buys_safe = buys_eval_df[(buys_eval_df['Beta']>1) & (buys_eval_df['Beta']<2) ].sort_values('Daily VaR',ascending=False)
print(f"{len(buys_safe)} Stocks:")
display(HTML(buys_safe.to_html(index=False)))
ret = df_to_html_file(buys_safe,"/var/www/html/home/viewable_pages/all_roads_lead_to_up_safe.php","All Roads Lead to UP & Safe") 

10 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,ADI,186.52,+/-1.98%,+/-$3.69,False,Buy,2023-06-12,Model2,"Buy,Buy,Buy",Buy,2022-12-23,"(50, 200)",1.2949,1.82,7.56,-4.73
2023-08-08,JBL,108.37,+/-1.99%,+/-$2.16,True,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2022-10-03,"(50, 200)",1.2523,7.19,7.67,-4.87
2023-08-08,ADBE,520.46,+/-2.34%,+/-$12.18,True,Buy,2023-05-19,Model2,"Buy,Buy,Buy",Buy,2023-03-21,"(50, 200)",1.4736,0.85,7.95,-5.65
2023-08-08,SWKS,108.72,+/-2.42%,+/-$2.63,False,Buy,2023-06-12,Model2,"Buy,Buy,Buy",Buy,2023-02-08,"(50, 200)",1.5268,-1.85,8.05,-5.72
2023-08-08,STX,67.12,+/-2.53%,+/-$1.7,False,Buy,2023-07-28,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(50, 200)",1.2141,1.19,7.58,-6.03
2023-08-08,MU,69.38,+/-2.69%,+/-$1.87,False,Buy,2023-07-26,Model2,"Buy,Buy,Buy",Buy,2023-03-09,"(50, 200)",1.5015,0.04,8.02,-6.38
2023-08-08,KLAC,498.68,+/-2.73%,+/-$13.63,False,Buy,2023-05-15,Model2,"Buy,Buy,Buy",Buy,2022-12-14,"(50, 200)",1.7169,4.23,8.48,-6.62
2023-08-08,AMAT,148.06,+/-2.84%,+/-$4.2,False,Buy,2023-05-15,Model2,"Buy,Buy,Buy",Buy,2023-01-10,"(50, 200)",1.8025,3.49,8.94,-6.86
2023-08-08,LRCX,697.38,+/-2.93%,+/-$20.4,False,Buy,2023-03-30,Model2,"Buy,Buy,Buy",Buy,2023-01-17,"(50, 200)",1.8100,2.63,8.96,-7.06
2023-08-08,ON,101.62,+/-3.26%,+/-$3.31,False,Buy,2023-05-01,Model1,"Buy,Buy,Buy",Buy,2022-08-25,"(50, 200)",1.9852,6.33,9.10,-7.96


### Save the Safe-Buys as Top Picks

In [32]:
buys_safe.to_csv("./top_picks.csv")

### High Sharpe Ratios (Top 15)

In [33]:
hi_sharpe_df = eval_df.sort_values(['%Sharpe Ratio'], ascending=False)
display(HTML(hi_sharpe_df.head(15).to_html(index=False)))
ret = df_to_html_file(hi_sharpe_df,"/var/www/html/home/viewable_pages/high_sharpe_ratios.php","High Sharpe Ratios") 

Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,SMCI,349.32,+/-3.74%,+/-$13.07,True,Buy,2023-07-10,Model2,"Sell,Sell,Sell",None,None,"(50, 200)",1.3871,11.23,7.98,-9.12
2023-08-08,JBL,108.37,+/-1.99%,+/-$2.16,True,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2022-10-03,"(50, 200)",1.2523,7.19,7.67,-4.87
2023-08-08,NVDA,446.38,+/-3.47%,+/-$15.47,True,Buy,2023-01-12,Model2,"Buy,Buy,Buy",Buy,2023-01-24,"(50, 200)",2.1795,6.51,9.82,-8.46
2023-08-08,ON,101.62,+/-3.26%,+/-$3.31,False,Buy,2023-05-01,Model1,"Buy,Buy,Buy",Buy,2022-08-25,"(50, 200)",1.9852,6.33,9.10,-7.96
2023-08-08,AVGO,882.81,+/-2.06%,+/-$18.23,True,Buy,2022-11-08,Model2,"Buy,Sell,Sell",Buy,2023-01-10,"(50, 200)",1.3460,5.52,7.68,-4.99
2023-08-08,ORCL,115.06,+/-1.8%,+/-$2.07,True,Sell,2023-06-26,Model2,"Buy,Buy,Buy",Buy,2022-12-09,"(50, 200)",0.8725,5.01,6.66,-4.32
2023-08-08,KLAC,498.68,+/-2.73%,+/-$13.63,False,Buy,2023-05-15,Model2,"Buy,Buy,Buy",Buy,2022-12-14,"(50, 200)",1.7169,4.23,8.48,-6.62
2023-08-08,PANW,210.98,+/-2.6%,+/-$5.48,True,Sell,2023-07-12,Model1,"Sell,Sell,Sell",Buy,2023-03-20,"(50, 200)",1.1747,4.07,7.30,-6.23
2023-08-08,CDNS,228.89,+/-2.24%,+/-$5.13,False,Buy,2023-05-18,Model2,"Sell,Sell,Sell",Buy,2023-01-10,"(50, 200)",1.3836,3.86,7.77,-5.41
2023-08-08,DE,431.31,+/-1.99%,+/-$8.6,True,Buy,2023-06-07,Model1,"Buy,Buy,Buy",Sell,2023-05-10,"(50, 200)",0.8351,3.57,6.57,-4.82


### UP the Next Day

In [34]:
LR_Next_Day_Recomm_only = eval_df[eval_df['LR Next_Day Recomm']=='Buy,Buy,Buy'].sort_values('Daily VaR',ascending=False)
print(f"{len(LR_Next_Day_Recomm_only)} Stocks:")
display(HTML(LR_Next_Day_Recomm_only.to_html(index=False)))
ret = df_to_html_file(LR_Next_Day_Recomm_only,"/var/www/html/home/viewable_pages/up_next_day.php","Linear Reg. Predicted Up_Next_Day_only") 

23 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,HON,187.73,+/-1.36%,+/-$2.56,False,Sell,2023-07-27,Model1,"Buy,Buy,Buy",Sell,2023-08-02,"(50, 200)",0.8163,-1.85,6.65,-3.20
2023-08-08,ORCL,115.06,+/-1.8%,+/-$2.07,True,Sell,2023-06-26,Model2,"Buy,Buy,Buy",Buy,2022-12-09,"(50, 200)",0.8725,5.01,6.66,-4.32
2023-08-08,AAPL,179.94,+/-1.82%,+/-$3.28,True,Sell,2023-08-04,Model2,"Buy,Buy,Buy",Buy,2023-03-23,"(50, 200)",1.2874,2.57,7.53,-4.37
2023-08-08,MSFT,330.11,+/-1.82%,+/-$6.02,False,Sell,2023-07-27,Model2,"Buy,Buy,Buy",Buy,2023-03-21,"(50, 200)",1.2624,3.41,7.69,-4.38
2023-08-08,ADI,186.52,+/-1.98%,+/-$3.69,False,Buy,2023-06-12,Model2,"Buy,Buy,Buy",Buy,2022-12-23,"(50, 200)",1.2949,1.82,7.56,-4.73
2023-08-08,DE,431.31,+/-1.99%,+/-$8.6,True,Buy,2023-06-07,Model1,"Buy,Buy,Buy",Sell,2023-05-10,"(50, 200)",0.8351,3.57,6.57,-4.82
2023-08-08,JBL,108.37,+/-1.99%,+/-$2.16,True,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2022-10-03,"(50, 200)",1.2523,7.19,7.67,-4.87
2023-08-08,ADBE,520.46,+/-2.34%,+/-$12.18,True,Buy,2023-05-19,Model2,"Buy,Buy,Buy",Buy,2023-03-21,"(50, 200)",1.4736,0.85,7.95,-5.65
2023-08-08,SWKS,108.72,+/-2.42%,+/-$2.63,False,Buy,2023-06-12,Model2,"Buy,Buy,Buy",Buy,2023-02-08,"(50, 200)",1.5268,-1.85,8.05,-5.72
2023-08-08,QCOM,117.82,+/-2.49%,+/-$2.94,False,Sell,2023-08-03,Model2,"Buy,Buy,Buy",Buy,2023-08-04,"(50, 200)",1.5288,-1.08,8.09,-5.89


### Down the Next Day

In [35]:
LR_Next_Day_Sell_only = eval_df[(eval_df['LR Next_Day Recomm']=='Sell,Buy,Buy') | 
                                (eval_df['LR Next_Day Recomm']=='Sell,Sell,Sell') | 
                                (eval_df['LR Next_Day Recomm']=='Sell,Buy,Sell') | 
                                (eval_df['LR Next_Day Recomm']=='Buy,Sell,Sell')]

print(f"{len(LR_Next_Day_Sell_only)} Stocks:")
display(HTML(LR_Next_Day_Sell_only.to_html(index=False)))
ret = df_to_html_file(LR_Next_Day_Sell_only,"/var/www/html/home/viewable_pages/down_next_day.php","Linear Reg. Predicted Down_Next_Day_only") 

18 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,QQQ,371.79,+/-1.56%,+/-$5.79,False,Buy,2023-03-21,Model2,"Buy,Sell,Sell",Buy,2023-03-13,"(50, 200)",1.2721,1.33,7.71,-3.72
2023-08-08,AMD,113.23,+/-3.29%,+/-$3.72,False,Sell,2023-06-21,Model2,"Sell,Sell,Sell",Buy,2023-03-16,"(50, 200)",1.9596,2.01,8.99,-7.90
2023-08-08,INTC,34.99,+/-2.29%,+/-$0.8,False,Buy,2023-07-28,Model2,"Sell,Sell,Sell",Buy,2023-05-09,"(50, 200)",1.2500,-2.14,7.47,-5.41
2023-08-08,META,312.49,+/-3.12%,+/-$9.74,True,Buy,2022-11-10,Model2,"Sell,Sell,Sell",Buy,2023-03-02,"(50, 200)",1.6822,1.71,8.66,-7.64
2023-08-08,IBM,145.74,+/-1.4%,+/-$2.04,False,Buy,2023-07-20,Model2,"Sell,Buy,Buy",Buy,2023-08-03,"(50, 200)",0.5387,1.47,5.93,-3.36
2023-08-08,TSLA,251.45,+/-3.81%,+/-$9.58,False,Sell,2023-07-20,Model2,"Sell,Sell,Sell",Buy,2023-06-23,"(50, 200)",1.8630,1.52,8.81,-9.19
2023-08-08,SPY,450.71,+/-1.16%,+/-$5.22,False,Buy,2023-03-31,Model2,"Sell,Buy,Buy",Buy,2023-02-02,"(50, 200)",1.0036,1.56,6.94,-2.75
2023-08-08,GOOGL,131.43,+/-2.05%,+/-$2.69,False,Buy,2023-07-14,Model2,"Sell,Buy,Buy",Buy,2023-05-03,"(50, 200)",1.3460,3.22,7.69,-4.93
2023-08-08,AMZN,142.22,+/-2.43%,+/-$3.46,False,Buy,2023-03-30,Model2,"Sell,Sell,Sell",Buy,2023-05-26,"(50, 200)",1.5091,-0.33,8.03,-5.77
2023-08-08,COIN,87.92,+/-5.94%,+/-$5.23,False,Sell,2023-08-04,Model2,"Sell,Buy,Buy",Buy,2023-06-30,"(50, 200)",2.7425,-1.11,2.59,-14.35


### Supertrend Winners and Still Supertrending

In [36]:
up_Supertrend = eval_df[(eval_df['Supertrend Result'] == 'Buy') & (eval_df['Supertrend Winner'] == True)].sort_values('ST Signal_Date', ascending=False)
print(f"{len(up_Supertrend)} Stocks:")
display(HTML(up_Supertrend.to_html(index=False)))
ret = df_to_html_file(up_Supertrend,"/var/www/html/home/viewable_pages/up_supertrend.php","Supertrend Winners and Still Supertrending") 

7 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,SMCI,349.32,+/-3.74%,+/-$13.07,True,Buy,2023-07-10,Model2,"Sell,Sell,Sell",None,None,"(50, 200)",1.3871,11.23,7.98,-9.12
2023-08-08,DE,431.31,+/-1.99%,+/-$8.6,True,Buy,2023-06-07,Model1,"Buy,Buy,Buy",Sell,2023-05-10,"(50, 200)",0.8351,3.57,6.57,-4.82
2023-08-08,ADBE,520.46,+/-2.34%,+/-$12.18,True,Buy,2023-05-19,Model2,"Buy,Buy,Buy",Buy,2023-03-21,"(50, 200)",1.4736,0.85,7.95,-5.65
2023-08-08,JBL,108.37,+/-1.99%,+/-$2.16,True,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2022-10-03,"(50, 200)",1.2523,7.19,7.67,-4.87
2023-08-08,NVDA,446.38,+/-3.47%,+/-$15.47,True,Buy,2023-01-12,Model2,"Buy,Buy,Buy",Buy,2023-01-24,"(50, 200)",2.1795,6.51,9.82,-8.46
2023-08-08,META,312.49,+/-3.12%,+/-$9.74,True,Buy,2022-11-10,Model2,"Sell,Sell,Sell",Buy,2023-03-02,"(50, 200)",1.6822,1.71,8.66,-7.64
2023-08-08,AVGO,882.81,+/-2.06%,+/-$18.23,True,Buy,2022-11-08,Model2,"Buy,Sell,Sell",Buy,2023-01-10,"(50, 200)",1.3460,5.52,7.68,-4.99


### Supertrending (Winners  or Not)

In [37]:
up_Supertrend = eval_df[eval_df['Supertrend Result'] == 'Buy'].sort_values('ST Signal_Date', ascending=False)
print(f"{len(up_Supertrend)} Stocks:")
display(HTML(up_Supertrend.to_html(index=False)))
ret = df_to_html_file(up_Supertrend,"/var/www/html/home/viewable_pages/up_supertrend_not_winners.php","Supertrending (Winners  or Not)") 


25 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,INTC,34.99,+/-2.29%,+/-$0.8,False,Buy,2023-07-28,Model2,"Sell,Sell,Sell",Buy,2023-05-09,"(50, 200)",1.2500,-2.14,7.47,-5.41
2023-08-08,STX,67.12,+/-2.53%,+/-$1.7,False,Buy,2023-07-28,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(50, 200)",1.2141,1.19,7.58,-6.03
2023-08-08,MU,69.38,+/-2.69%,+/-$1.87,False,Buy,2023-07-26,Model2,"Buy,Buy,Buy",Buy,2023-03-09,"(50, 200)",1.5015,0.04,8.02,-6.38
2023-08-08,IBM,145.74,+/-1.4%,+/-$2.04,False,Buy,2023-07-20,Model2,"Sell,Buy,Buy",Buy,2023-08-03,"(50, 200)",0.5387,1.47,5.93,-3.36
2023-08-08,GOOGL,131.43,+/-2.05%,+/-$2.69,False,Buy,2023-07-14,Model2,"Sell,Buy,Buy",Buy,2023-05-03,"(50, 200)",1.3460,3.22,7.69,-4.93
2023-08-08,SMCI,349.32,+/-3.74%,+/-$13.07,True,Buy,2023-07-10,Model2,"Sell,Sell,Sell",None,None,"(50, 200)",1.3871,11.23,7.98,-9.12
2023-08-08,ADI,186.52,+/-1.98%,+/-$3.69,False,Buy,2023-06-12,Model2,"Buy,Buy,Buy",Buy,2022-12-23,"(50, 200)",1.2949,1.82,7.56,-4.73
2023-08-08,SWKS,108.72,+/-2.42%,+/-$2.63,False,Buy,2023-06-12,Model2,"Buy,Buy,Buy",Buy,2023-02-08,"(50, 200)",1.5268,-1.85,8.05,-5.72
2023-08-08,DE,431.31,+/-1.99%,+/-$8.6,True,Buy,2023-06-07,Model1,"Buy,Buy,Buy",Sell,2023-05-10,"(50, 200)",0.8351,3.57,6.57,-4.82
2023-08-08,DIA,354.62,+/-0.98%,+/-$3.49,False,Buy,2023-06-02,Model2,"Sell,Buy,Buy",Buy,2022-12-14,"(50, 200)",0.7994,1.03,6.61,-2.34


### Fast SMA Crossed Slow SMA

In [38]:
Crossed_up = eval_df[eval_df['SMA Crossed_Up'] == 'Buy'].sort_values('SMA_X_Date', ascending=False)
print(f"{len(Crossed_up)} Stocks")
display(HTML(Crossed_up.to_html(index=False)))
ret = df_to_html_file(Crossed_up,"/var/www/html/home/viewable_pages/crossed_up.php","Fast SMA Crossed Slow SMA") 

38 Stocks


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,QCOM,117.82,+/-2.49%,+/-$2.94,False,Sell,2023-08-03,Model2,"Buy,Buy,Buy",Buy,2023-08-04,"(50, 200)",1.5288,-1.08,8.09,-5.89
2023-08-08,IBM,145.74,+/-1.4%,+/-$2.04,False,Buy,2023-07-20,Model2,"Sell,Buy,Buy",Buy,2023-08-03,"(50, 200)",0.5387,1.47,5.93,-3.36
2023-08-08,SQ,62.60,+/-4.33%,+/-$2.71,False,Sell,2023-08-04,Model2,"Buy,Buy,Buy",Buy,2023-07-31,"(50, 200)",2.5533,-2.89,10.30,-10.23
2023-08-08,TXN,166.82,+/-1.8%,+/-$3.0,False,Sell,2023-08-02,Model2,"Sell,Sell,Sell",Buy,2023-07-13,"(50, 200)",1.1784,0.05,7.33,-4.28
2023-08-08,COIN,87.92,+/-5.94%,+/-$5.23,False,Sell,2023-08-04,Model2,"Sell,Buy,Buy",Buy,2023-06-30,"(50, 200)",2.7425,-1.11,2.59,-14.35
2023-08-08,TSLA,251.45,+/-3.81%,+/-$9.58,False,Sell,2023-07-20,Model2,"Sell,Sell,Sell",Buy,2023-06-23,"(50, 200)",1.8630,1.52,8.81,-9.19
2023-08-08,MRVL,63.55,+/-3.57%,+/-$2.27,False,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2023-06-01,"(50, 200)",2.1622,2.46,9.45,-8.46
2023-08-08,AMZN,142.22,+/-2.43%,+/-$3.46,False,Buy,2023-03-30,Model2,"Sell,Sell,Sell",Buy,2023-05-26,"(50, 200)",1.5091,-0.33,8.03,-5.77
2023-08-08,STX,67.12,+/-2.53%,+/-$1.7,False,Buy,2023-07-28,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(50, 200)",1.2141,1.19,7.58,-6.03
2023-08-08,INTC,34.99,+/-2.29%,+/-$0.8,False,Buy,2023-07-28,Model2,"Sell,Sell,Sell",Buy,2023-05-09,"(50, 200)",1.2500,-2.14,7.47,-5.41


### SMA Crossed and in Supertrend

In [39]:
Crossed_up = eval_df[(eval_df['SMA Crossed_Up'] == 'Buy') & (eval_df['Supertrend Result'] == 'Buy') ].sort_values('SMA_X_Date', ascending=False)
print(f"{len(Crossed_up)} Stocks:")
# display(Crossed_up.hide_index())
display(HTML(Crossed_up.to_html(index=False)))
ret = df_to_html_file(Crossed_up,"/var/www/html/home/viewable_pages/crossed_up_supertrend.php","SMA Crossed and in Supertrend") 

23 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,IBM,145.74,+/-1.4%,+/-$2.04,False,Buy,2023-07-20,Model2,"Sell,Buy,Buy",Buy,2023-08-03,"(50, 200)",0.5387,1.47,5.93,-3.36
2023-08-08,MRVL,63.55,+/-3.57%,+/-$2.27,False,Buy,2023-05-17,Model2,"Buy,Buy,Buy",Buy,2023-06-01,"(50, 200)",2.1622,2.46,9.45,-8.46
2023-08-08,AMZN,142.22,+/-2.43%,+/-$3.46,False,Buy,2023-03-30,Model2,"Sell,Sell,Sell",Buy,2023-05-26,"(50, 200)",1.5091,-0.33,8.03,-5.77
2023-08-08,STX,67.12,+/-2.53%,+/-$1.7,False,Buy,2023-07-28,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(50, 200)",1.2141,1.19,7.58,-6.03
2023-08-08,INTC,34.99,+/-2.29%,+/-$0.8,False,Buy,2023-07-28,Model2,"Sell,Sell,Sell",Buy,2023-05-09,"(50, 200)",1.2500,-2.14,7.47,-5.41
2023-08-08,GOOGL,131.43,+/-2.05%,+/-$2.69,False,Buy,2023-07-14,Model2,"Sell,Buy,Buy",Buy,2023-05-03,"(50, 200)",1.3460,3.22,7.69,-4.93
2023-08-08,PLTR,16.89,+/-4.41%,+/-$0.74,False,Buy,2023-05-09,Model1,"Sell,Sell,Sell",Buy,2023-04-05,"(50, 200)",1.8411,0.45,8.76,-10.55
2023-08-08,ADBE,520.46,+/-2.34%,+/-$12.18,True,Buy,2023-05-19,Model2,"Buy,Buy,Buy",Buy,2023-03-21,"(50, 200)",1.4736,0.85,7.95,-5.65
2023-08-08,QQQ,371.79,+/-1.56%,+/-$5.79,False,Buy,2023-03-21,Model2,"Buy,Sell,Sell",Buy,2023-03-13,"(50, 200)",1.2721,1.33,7.71,-3.72
2023-08-08,MU,69.38,+/-2.69%,+/-$1.87,False,Buy,2023-07-26,Model2,"Buy,Buy,Buy",Buy,2023-03-09,"(50, 200)",1.5015,0.04,8.02,-6.38


### Test One or a Group of Stocks

In [40]:
stock_to_test = 'swks'
single_stock_df = init_eval_table()
single_stock_list = [stock_to_test]
single_stock_list = ['jbl','aapl','nvda','msft','googl','amzn','nflx','meta','adbe','cdns','de','avgo','orcl']
single_stock_df = recommendation_table(single_stock_df,single_stock_list, lookback_years=5, sma_fast=50, sma_slow=200,run_update_models=False)
display(HTML(single_stock_df.sort_values('SMA_X_Date', ascending=False).to_html(index=False)))

Performing Analysis and Recommendations ...

JBL,AAPL,NVDA,MSFT,GOOGL,AMZN,NFLX,META,ADBE,CDNS,DE,AVGO,ORCL,Done!


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,DE,430.08,+/-2.12%,+/-$9.11,True,Buy,2023-06-07,N/A,"Buy,Buy,Buy",Buy,2023-08-08,"(50, 200)",1.0245,3.65,11.14,-5.13
2023-08-08,AMZN,142.22,+/-2.25%,+/-$3.21,False,Buy,2023-03-30,N/A,"Sell,Sell,Sell",Buy,2023-05-26,"(50, 200)",1.1173,3.12,11.79,-5.43
2023-08-08,GOOGL,131.46,+/-1.99%,+/-$2.61,False,Buy,2023-07-14,N/A,"Buy,Buy,Buy",Buy,2023-05-03,"(50, 200)",1.1569,3.33,12.05,-4.78
2023-08-08,AAPL,179.98,+/-2.04%,+/-$3.68,True,Sell,2023-08-04,N/A,"Buy,Buy,Buy",Buy,2023-03-23,"(50, 200)",1.2241,5.21,12.50,-4.96
2023-08-08,MSFT,326.24,+/-1.94%,+/-$6.32,False,Sell,2023-07-27,N/A,"Buy,Buy,Buy",Buy,2023-03-21,"(50, 200)",1.2186,5.01,12.47,-4.69
2023-08-08,ADBE,529.73,+/-2.34%,+/-$12.38,True,Buy,2023-05-19,N/A,"Buy,Sell,Sell",Buy,2023-03-21,"(50, 200)",1.2972,3.75,13.01,-5.66
2023-08-08,META,312.83,+/-2.75%,+/-$8.6,True,Buy,2022-11-10,N/A,"Sell,Sell,Sell",Buy,2023-03-02,"(50, 200)",1.2812,2.18,13.04,-6.71
2023-08-08,NVDA,446.56,+/-3.29%,+/-$14.68,True,Buy,2023-01-12,N/A,"Buy,Buy,Buy",Buy,2023-01-24,"(50, 200)",1.7479,5.76,16.06,-8.05
2023-08-08,CDNS,229.05,+/-2.24%,+/-$5.13,True,Buy,2023-05-18,N/A,"Sell,Sell,Sell",Buy,2023-01-10,"(50, 200)",1.2207,5.71,12.48,-5.44
2023-08-08,AVGO,883.39,+/-2.31%,+/-$20.44,True,Buy,2022-11-08,N/A,"Buy,Sell,Sell",Buy,2023-01-10,"(50, 200)",1.2750,4.11,12.85,-5.62


### Filter the Group

In [41]:
single_stock_df= screen_for_buys(eval_df=single_stock_df,ignore_supertrend_winners=True)
display(HTML(single_stock_df.sort_values('SMA_X_Date', ascending=False).to_html(index=False)))

Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2023-08-08,DE,430.08,+/-2.12%,+/-$9.11,True,Buy,2023-06-07,N/A,"Buy,Buy,Buy",Buy,2023-08-08,"(50, 200)",1.0245,3.65,11.14,-5.13
2023-08-08,GOOGL,131.46,+/-1.99%,+/-$2.61,False,Buy,2023-07-14,N/A,"Buy,Buy,Buy",Buy,2023-05-03,"(50, 200)",1.1569,3.33,12.05,-4.78
2023-08-08,NVDA,446.56,+/-3.29%,+/-$14.68,True,Buy,2023-01-12,N/A,"Buy,Buy,Buy",Buy,2023-01-24,"(50, 200)",1.7479,5.76,16.06,-8.05
2023-08-08,JBL,108.20,+/-2.34%,+/-$2.53,True,Buy,2023-05-17,N/A,"Buy,Buy,Buy",Buy,2022-10-03,"(50, 200)",1.2752,4.70,12.85,-5.70


In [42]:
import requests
import textwrap
nasdaq_api_key = 'yYCEL8BqzxYUsgG67FTb'

def get_company_info(ticker_symbol):
    ticker_symbol = str(ticker_symbol).upper()
    # Set the endpoint URL and the API key
    url = "https://api.nasdaq.com/api/company/{}/company-profile"
    api_key = nasdaq_api_key

    # Format the endpoint URL with the ticker symbol
    endpoint_url = url.format(ticker_symbol)

    # Set the headers and parameters for the request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299",
        "Accept-Language": "en-US,en;q=0.9"
    }

    params = {
        "apikey": nasdaq_api_key
    }

    # Send the request and get the response
    response = requests.get(endpoint_url, headers=headers, params=params)

    # Parse the response JSON
    response_json = response.json()

    # Get the company full name and sector
    dic = {
        "Ticker" : ticker_symbol,
        "Full_Name" : response_json["data"]["CompanyName"]['value'],
        "Sector" : response_json["data"]["Sector"]["value"],
        "Industry" : response_json["data"]["Industry"]["value"],
        "Region" : response_json["data"]["Region"]["value"],
        "Address" : response_json["data"]["Address"]["value"],
        "Description" : response_json["data"]["CompanyDescription"]["value"]
        }

    df = pd.DataFrame.from_dict(dic, orient='index').T
    return df

company_info = get_company_info('CHTR')
print(f"Company  : {company_info.loc[0]['Full_Name']}")
print(f"Sector   : {company_info.loc[0]['Sector']}")
print(f"Industry : {company_info.loc[0]['Industry']}")
print(f"Region   : {company_info.loc[0]['Region']}")

wrapped_text = textwrap.wrap(company_info.loc[0]['Description'],100)
for line in wrapped_text:
    print(line)


Company  : Charter Communications, Inc.
Sector   : Telecommunications
Industry : Cable & Other Pay Television Services
Region   : North America
Charter is the product of the 2016 merger of three cable companies, each with a decades-long history
in the business: Legacy Charter, Time Warner Cable, and Bright House Networks. The firm now holds
networks capable of providing television, internet access, and phone services to roughly 54 million
U.S. homes and businesses, around 40% of the country. Across this footprint, Charter serves 29
million residential and 2 million commercial customer accounts under the Spectrum brand, making it
the second-largest U.S. cable company behind Comcast. The firm also owns, in whole or in part,
sports and news networks, including Spectrum SportsNet (long-term local rights to Los Angeles Lakers
games), SportsNet LA (Los Angeles Dodgers), SportsNet New York (New York Mets), and Spectrum News
NY1.


In [43]:
import os
import json

def build_sp500_companies_database():
    n, m, sp_df = load_sp500_list()
    stocks_sp500 = sp_df.Symbol.values
    stocks_sp500 = [s.strip().upper() for s in list(stocks_sp500)]
    
    if not os.path.exists("./company_info"):
        os.makedirs("./company_info")
    
    for s in stocks_sp500:
        company_info = get_company_info(s)
        filename = f"./company_info/{s}.json"
        
        with open(filename, "w") as outfile:
            json.dump(company_info, outfile, indent=4)
        
        print(f"Saved {filename}")
    
    print("Done!")

# build_sp500_companies_database()

In [44]:
sp_df.tail(50)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
446,TXN,Texas Instruments,Information Technology,Semiconductors,"Dallas, Texas",NaN,97476,1930
447,TXT,Textron,Industrials,Aerospace & Defense,"Providence, Rhode Island",1978-12-31,217346,1923
456,TYL,Tyler Technologies,Information Technology,Application Software,"Plano, Texas",2020-06-22,860731,1966
462,UAL,United Airlines Holdings,Industrials,Passenger Airlines,"Chicago, Illinois",2015-09-03,100517,1967
459,UDR,"UDR, Inc.",Real Estate,Residential REITs,"Highlands Ranch, Colorado",2016-03-07,74208,1972
466,UHS,Universal Health Services,Health Care,Health Care Facilities,"King of Prussia, Pennsylvania",2014-09-20,352915,1979
460,ULTA,Ulta Beauty,Consumer Discretionary,Specialty Stores,"Bolingbrook, Illinois",2016-04-18,1403568,1990
465,UNH,UnitedHealth Group,Health Care,Managed Health Care,"Minnetonka, Minnesota",1994-07-01,731766,1977
461,UNP,Union Pacific Corporation,Industrials,Rail Transportation,"Omaha, Nebraska",1957-03-04,100885,1862
463,UPS,United Parcel Service,Industrials,Air Freight & Logistics,"Sandy Springs, Georgia",2002-07-22,1090727,1907
